<a href="https://colab.research.google.com/github/mikelalvarezbejarano/sis420/blob/main/pregunta2redneufin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [ ]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [ ]:
#funciones de activacion
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Optimizador, stocastic greadent descent
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [ ]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

#Carga de dataset
data = pd.read_csv('/content/drive/MyDrive/dataset/univer/Top-Largest-Universities.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Rank                  84 non-null     int64 
 1   Institution           84 non-null     object
 2   Location              84 non-null     object
 3   Continent             84 non-null     object
 4   Founded               84 non-null     object
 5   Affiliation           84 non-null     object
 6   Distance / In-Person  84 non-null     object
 7   Enrollment            84 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 5.4+ KB


In [ ]:
columnas_categoricas = data.select_dtypes(include=['object']).columns
columnas_fechas = data.select_dtypes(include=['datetime64']).columns

#Procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

#Fechas a numeros
for columna in columnas_fechas:
  data[columna] = pd.to_numeric(data[columna].map(datetime.timestamp))

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Rank                  84 non-null     int64
 1   Institution           84 non-null     int64
 2   Location              84 non-null     int64
 3   Continent             84 non-null     int64
 4   Founded               84 non-null     int64
 5   Affiliation           84 non-null     int64
 6   Distance / In-Person  84 non-null     int64
 7   Enrollment            84 non-null     int64
dtypes: int64(8)
memory usage: 5.4 KB


In [ ]:
columnas_eiliminar = ['ID']
data = data.drop(columnas_eiliminar, axis=1)

In [ ]:
# data.info()

In [ ]:
print(data)

    Rank  Institution  Location  Continent  Founded  Affiliation  \
0      1            0        45          1       54            1   
1      2            8         5          4       42            1   
2      3           33        13          1       59            1   
3      4            4        10          1       38            1   
4      5            3        24          1       48            1   
..   ...          ...       ...        ...      ...          ...   
79    80           73        64          4       11            1   
80    81           12        34          2        0            1   
81    82           15        23          1       67            1   
82    83           64        18          2       48            1   
83    84            1        23          1       51            0   

    Distance / In-Person  Enrollment  
0                      3     7140000  
1                      4     2133846  
2                      4     2097182  
3                      3   

In [ ]:
x = data.iloc[:, :20]
y = data.iloc[:, 20]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print('---'*20)
print(y)

IndexError: ignored

In [ ]:
# Crea un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('smoking2.csv', index=False)

In [ ]:
data = np.loadtxt("/content/smoking2.csv", delimiter=',',skiprows=1)
# print(data)
x, y = data[:, :25].astype(int), data[:, 0].astype(int)
x = x.reshape(len(x),25)
print(x.shape)
print(y.shape)

(55692, 25)
(55692,)


In [ ]:
x_train, x_test, y_train, y_test = x[:800] , x[800:] , y[:800].astype(np.int64), y[800:].astype(np.int64)
x_train.shape , x_test.shape

y_train.shape , y_test.shape

((800,), (54892,))

In [ ]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [ ]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [ ]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

In [ ]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [ ]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [ ]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [ ]:
model = MLPBinaryClassification(D_in=25, H=2, D_out=1)
epochs, lr = 60, 0.04

# normalización datos
x_mean = np.mean(x_train)
x_std = np.std(x_train)
x_std = np.nan_to_num(x_std, nan=1.0)


# Calcula x_norm
x_norm = (x_train - x_mean) / x_std

model.fit(x_norm, y_train, epochs, lr, batch_size=1, log_each=10)

Epoch: 10/60, Loss: -0.06444
Epoch: 20/60, Loss: -0.06579
Epoch: 30/60, Loss: -0.06624
Epoch: 40/60, Loss: -0.06647
Epoch: 50/60, Loss: -0.06660
Epoch: 60/60, Loss: -0.06669


In [ ]:
# últimos pesos encontrados

w = model.ws[-1]
w

(array([[ 0.04400106, -0.20151279],
        [-0.35429604, -0.27128709],
        [ 0.06958799, -0.14705891],
        [-0.0161799 ,  0.06227872],
        [ 0.1940286 , -0.21758662],
        [ 0.1647537 , -0.00374944],
        [-0.24959116,  0.02432774],
        [-0.02139168,  0.18908866],
        [ 0.07153555,  0.43891658],
        [-0.27128323, -0.39182419],
        [-0.24059502,  0.09181556],
        [ 0.25161644, -0.36458946],
        [-0.37574668,  0.13236993],
        [-0.38014667, -0.00923061],
        [-0.06929962,  0.53501529],
        [-0.61324315,  0.33436185],
        [-0.30254587,  0.53141198],
        [ 0.08174561,  0.20702509],
        [-0.5032847 , -0.03355023],
        [ 0.50677931, -0.28064337],
        [-0.37088008,  0.43722063],
        [-0.20169382,  0.08341795],
        [ 0.19653769, -0.12915138],
        [ 0.34405547, -0.32260929],
        [ 0.05122259,  0.27120063]]),
 array([-0.11441021, -0.03861926]),
 array([[-1.70735352],
        [-0.74384358]]),
 array([0.4338

In [ ]:
print("Primeros 5 datos de entrada de prueba (X_test):")
print(x_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (X_test):
[[  1  30 165  65  84   0   1   1   1 100  70  88 159  98  78  61  15   1
    0  30  24  28   0   0   1]
 [  1  35 165  75  94   1   1   1   1 120  80  98 208 247  53 106  17   3
    1  35  49 141   0   0   0]
 [  1  40 180  65  77   1   1   1   1 126  76  92 132  80  62  54  14   1
    0  19  18  87   0   0   1]
 [  0  60 155  60  78   0   0   1   1 132  82  90 178 138  46 103  12   1
    0  37  20  22   0   0   1]
 [  0  70 145  55  88   0   0   1   1 129  74 147 149 170  52  63  11   1
    1  17  14  15   0   0   0]]
Primeras 5 etiquetas de prueba (y_test):
[1 1 1 0 0]


In [ ]:
# nuevo punto
x_new = x_test
X_new = [0, 70, 145, 55, 88, 0, 0, 1, 1, 129, 74, 147, 149, 170, 52, 63, 11, 1, 1, 17, 14, 15, 0, 0, 0]
y_pred = model.predict(w, X_new)
y_pred

array([0.60680309])

In [ ]:
def evaluate(perceptron, x, t = 0.5):
    w = perceptron.ws[-1]
    x = np.c_[np.ones(len(x)), x]
    y = perceptron(w, x)
    return (y > t).astype(np.int64)

In [ ]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)